In [207]:
#import libraries 
import pandas as pd
import pyreadstat
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import html

In [208]:
#open file 
file = '/Users/utente/Library/CloudStorage/OneDrive-Raccoltecondivise-INTWIGsrl/Archivio - Documenti/Lavori/OMNIBUS/2023/Wave 01/05. Analisi/Profilazione/omnibus_finalefonte_peso con R.sav'
df = pd.read_spss(file)
df['id'] = range(1, len(df) +1)

In [209]:
#select columns of interest 
columns_of_interest = df.iloc[:, 14:115]
ids = df.iloc[:, 0]
result = pd.concat([ids, columns_of_interest], axis=1)

In [210]:
result['id'] = range(1, len(result) +1)

In [211]:
#refactor the variables 
result['aut'] = result['aut'].replace(["Estrema sinistra", "Sinistra", "Centro sinistra", "Centro", "Centro destra", "Destra", "Estrema destra", "Non so"], [1, 2, 3,4, 5,6,7,8])

for col in result.columns:
    
    if 'INDCOL' in col:
        variable = 'INDCOL'
        result[col] = result[col].replace(['Molto', 'Abbastanza', 'Poco', 'Per niente'], [4, 3, 2,1])
        
    elif 'CONS' in col:
        variable = 'CONS'
        result[col] = result[col].replace(["Molto d'accordo", "Abbastanza d'accordo", "Non molto d'accordo", "Per niente d'accordo"], [4, 3, 2,1])
        
    elif 'PERS' in col:
        variable = 'PERS'
        result[col] = result[col].replace(["Molto d'accordo", "Abbastanza d'accordo", "Non molto d'accordo", "Per niente d'accordo"], [4, 3, 2,1])
        
    elif 'SOC' in col:
        variable = 'SOC'
        result[col] = result[col].replace(["Andare a divertirmi nei locali più frequentati.", "Uscire con gli amici per bere e mangiare qualcosa insieme.", "Rimanere a casa la sera a guardare un film con la mia famiglia, il mio partner o i miei amici.", "Andare in posti poco frequentati per godermi di più le persone a cui voglio bene.", "Non fare tardi la sera per essere più riposati al mattino."], [1, 2, 3,4, 5])
        
    elif 'spesa1_1' in col or 'spesa1_2'in col:
        if col == 'spesa1_1':
            variable = 'spesa1_1'
            result[variable] = result[variable].replace(["Supermercato", 'Negozio di quartiere (es. macellaio, fruttivendolo, ecc.)', 'Online', 'Online con servizio a domicilio', 'Online con opzione di ritiro in loco', 'Online con opzione di ritiro al supermercato', 'Mercato'], [1, 2, 3,3, 4, 4, 5])
            result[variable] = result[variable].cat.add_categories([0])
            result[variable] = result[variable].fillna(0)
            
        elif col == 'spesa1_2':
            variable = 'spesa1_2'
    
            result[variable] = result[variable].replace(["Supermercato", 'Negozio di quartiere (es. macellaio, fruttivendolo, ecc.)', 'Online', 'Online con servizio a domicilio', 'Online con opzione di ritiro in loco', 'Online con opzione di ritiro al supermercato', 'Mercato'], [1, 2, 3,3, 4, 4, 5])
            result[variable] = result[variable].cat.add_categories([0])
            result[variable] = result[variable].fillna(0)
    
    
    elif 'T1' in col:
        result[col] = result[col].replace(["Molto d'accordo", "Abbastanza d'accordo", "Non molto d'accordo", "Per niente d'accordo"], [4, 3, 2,1])
        
        
    elif 'T2' in col:
        result[col] = result[col].replace(["Sì, e l’ho utilizzata", "Sì, so cosa è", "Ne ho sentito parlare ma non so cosa è", "Mai sentito parlare"], [1,2,3,4])
        
        
        

In [212]:
#I define the group of variables within which we can identify some patterns 
pers_cols = pd.concat([df.iloc[:, 0], result.filter(regex='^PERS', axis=1)], axis=1)
ind_cols = pd.concat([df.iloc[:, 0], result.filter(regex='^INDCOL', axis=1)], axis=1)
cons_cols = pd.concat([df.iloc[:, 0],result.filter(regex='^CONS', axis=1)], axis=1)
tech_cols = pd.concat([df.iloc[:, 0], result.filter(regex='^T1', axis=1)], axis=1)
tech_conoscenza = pd.concat([df.iloc[:, 0], result.filter(regex='^T2', axis=1)], axis=1)
spesa_cols = pd.concat([df.iloc[:, 0], result.filter(regex = '^(spesa1_1|spesa1_2)', axis = 1)], axis = 1)

### If you suspect that participants are answering randomly...
#### you could calculate the standard deviation of their answers and compare it to a threshold value.

If the standard deviation is above the threshold, it could indicate that the participant is answering randomly. 

In [226]:
#function to calculate the standard deviation

def check_std(dataset, threshold = 0.2):
    ids_problematici = []

    df_copy = dataset.copy(deep = True)

    df_copy['std'] = dataset.iloc[:,1:len(dataset)].std(axis=1)

    
    for index, row in df_copy.iterrows():

        if row['std'] < threshold:
            ids_problematici.append(int(row['id']))
    
    return set(ids_problematici)

In [230]:
#calculate the std for each group of variable and identify the problematic users 
list1 = check_std(pers_cols)
print(f'Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sulla PERSONALITÀ sono: {list1}')
print()
list2 = check_std(ind_cols)
print(f'Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria INDIVIDUALISMO-COLLETTIVISMO sono: {list2}')
print()
list3 = check_std(cons_cols)
print(f'Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sul CONSUMO sono: {list3}')
print()
list4 = check_std(tech_cols)
print(f'Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sulla TECNOLOGIA sono: {list4}')
print()
list5 = check_std(tech_conoscenza)
print(f'Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sulla tecnologia conoscenza sono: {list1}')

Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sulla PERSONALITÀ sono: {481, 388, 101, 485, 39, 558, 506, 571, 159}

Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria INDIVIDUALISMO-COLLETTIVISMO sono: {481, 451, 39, 168, 552, 426, 366, 495, 558, 307, 310, 407, 120, 571, 570, 411, 159}

Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sul CONSUMO sono: {260, 388, 135, 137, 150, 152, 281, 537, 283, 159, 416, 36, 293, 39, 168, 312, 571, 195, 451, 72, 200, 208, 338, 84, 85, 352, 481, 495, 372, 500, 509}

Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sulla TECNOLOGIA sono: {388, 279, 538, 283, 411, 159, 39, 558, 312, 570, 571, 60, 62, 451, 200, 75, 77, 338, 84, 85, 344, 216, 481, 355, 376, 229, 366, 495, 241, 370, 248}

Gli ID che hanno un deviazione standard inferiore a 0.2 nella batteria sulla tecnologia conoscenza sono: {481, 388, 101, 485, 39, 558, 506, 571, 159}


In [235]:
#intersecate the set of problematic users and find the shared ones 
common_12 = list1 & list2
common_13 = list1 & list3
common_elements = list1 & list2 & list3 &list4 & list5
print(f'The IDs of problematic users which have suspicious answers over multiple batteries of valiables are: {common_elements}')

{481, 388, 39, 571, 159}
The IDs of problematic users which have suspicious answers over multiple batteries of valiables are: {571, 159}


### We could calculate the frequency of each answer choice 
(e.g., how many times a participant answers 1 or 4, which are the extremes in the Likert scale) and look for patterns in those frequencies.

In [236]:
#report the frequency of 1 and 4, as answers and return it as a value

def check_frequency(dataset, t_1 = 6, t_4 = 6):
    
    ids_problematici_1 = []
    ids_problematici_4 = []
    
    df_copy = dataset.copy(deep = True)

    df_copy['frequency_1'] = df_copy.iloc[:,1:len(dataset)].apply(lambda x: x.value_counts().get(1, 0), axis=1)
    df_copy['frequency_4'] = df_copy.iloc[:,1:len(dataset)].apply(lambda x: x.value_counts().get(4, 0), axis=1)
    df_copy['pattern'] = [list(row) for i, row in pers_cols.iloc[:,1:].iterrows()]
     

    for index, row in df_copy.iterrows():

        if row['frequency_1'] >= t_1:
            ids_problematici_1.append((row['id'], row['frequency_1']))
            
        if row['frequency_4'] >= t_4:
            ids_problematici_4.append((row['id'], row['frequency_4']))
    
    return df_copy, ids_problematici_1, ids_problematici_4

In [237]:
dataset_enriched, freq_1, freq_4 = check_frequency(pers_cols)

In [238]:
dataset_enriched[dataset_enriched['id'] == 540]

,id,PERS_1,PERS_2,PERS_3,PERS_4,PERS_5,PERS_6,PERS_7,PERS_8,PERS_9,PERS_10,frequency_1,frequency_4,pattern
539,540,4,1,1,1,4,1,1,3,4,1,6,3,"[4, 1, 1, 1, 4, 1, 1, 3, 4, 1]"


In [239]:
#this is the dataset enriched with the new informations
dataset_enriched

,id,PERS_1,PERS_2,PERS_3,PERS_4,PERS_5,PERS_6,PERS_7,PERS_8,PERS_9,PERS_10,frequency_1,frequency_4,pattern
0,1,4,3,3,1,4,2,3,3,3,2,1,2,"[4, 3, 3, 1, 4, 2, 3, 3, 3, 2]"
1,2,4,1,4,1,4,4,3,3,1,3,3,4,"[4, 1, 4, 1, 4, 4, 3, 3, 1, 3]"
2,3,4,3,4,2,3,3,2,3,4,3,0,3,"[4, 3, 4, 2, 3, 3, 2, 3, 4, 3]"
3,4,3,2,4,1,3,2,3,2,3,2,1,1,"[3, 2, 4, 1, 3, 2, 3, 2, 3, 2]"
4,5,4,2,3,3,3,3,2,3,3,3,0,1,"[4, 2, 3, 3, 3, 3, 2, 3, 3, 3]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,583,3,3,4,1,2,3,1,4,4,2,2,3,"[3, 3, 4, 1, 2, 3, 1, 4, 4, 2]"
583,584,3,3,1,4,4,3,2,2,4,2,1,3,"[3, 3, 1, 4, 4, 3, 2, 2, 4, 2]"
584,585,2,3,2,3,4,1,3,2,3,2,1,1,"[2, 3, 2, 3, 4, 1, 3, 2, 3, 2]"
585,586,3,2,3,1,3,2,1,3,3,1,3,0,"[3, 2, 3, 1, 3, 2, 1, 3, 3, 1]"


In [240]:
ind_col_dataset, freq1, freq4 = check_frequency(ind_cols)
print(freq1)
print(freq4)

[(571, 7)]
[(318, 6), (451, 7), (552, 7)]


In [241]:
ind_col_dataset[ind_col_dataset['id'] == 552]

,id,INDCOL_1,INDCOL_2,INDCOL_3,INDCOL_4,INDCOL_5,INDCOL_6,INDCOL_7,frequency_1,frequency_4,pattern
551,552,4,4,4,4,4,4,4,0,7,"[1, 4, 1, 4, 4, 4, 4, 1, 4, 2]"


### The code below helps us identify again those which gave the same answer to many questions. It is implemented in another way, but we are controlling for the same condition as before.

In [247]:
# define the threshold for what is considered suspicious
def pattern_identification(enriched_dataset):
    threshold = 8

    #initialize variables to keep track of the current number and the count
    current_number = None
    count = 0
    ids = []
    
    # loop over the list of numbers
    for i, row in pd.DataFrame(dataset_enriched).iterrows():
        for number in row[13]:
            if number == current_number:
                
                # increment the count if the number is the same as the previous one
                count += 1
                if count >= threshold:
                    
                    # print a message if the count exceeds the threshold
                    print(f"Suspicious pattern: {count} in this row {row[13]}, with id = {row[0]}")
                    ids.append(row[0])
            else:
                # reset the count if the number is different from the previous one
                current_number = number
                count = 1


    return set(ids)


In [248]:
to_be_eliminated = list(sorted(pattern_identification(dataset_enriched)))

Suspicious pattern: 8 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 39
Suspicious pattern: 9 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 39
Suspicious pattern: 10 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 39
Suspicious pattern: 8 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 101
Suspicious pattern: 9 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 101
Suspicious pattern: 10 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 101
Suspicious pattern: 8 in this row [2, 2, 2, 2, 2, 2, 2, 2, 2, 2], with id = 159
Suspicious pattern: 9 in this row [2, 2, 2, 2, 2, 2, 2, 2, 2, 2], with id = 159
Suspicious pattern: 10 in this row [2, 2, 2, 2, 2, 2, 2, 2, 2, 2], with id = 159
Suspicious pattern: 8 in this row [3, 3, 3, 3, 3, 3, 3, 4, 3, 3], with id = 213
Suspicious pattern: 9 in this row [3, 3, 3, 3, 3, 3, 3, 4, 3, 3], with id = 213
Suspicious pattern: 10 in this row [3, 3, 3, 3, 3, 3, 3, 4, 3, 3], with id = 213
Suspicious pattern: 8 in this row [3, 3

In [244]:
print(f'The set of IDs corresponding to problematic responders are: {to_be_eliminated}')

The set of IDs corresponding to problematic responders are: [39, 101, 159, 213, 326, 388, 389, 481, 485, 495, 506, 558, 571, 572]


### With this function we are able to check if there is a diagonal pattern in filling in the survey. 

In [245]:
def has_diagonal_mode(lst):
    
    # Split the list into sub-lists
    sub_lists = [lst[i:i+5] for i in range(0, len(lst), 5)]
    #print(sub_lists)

    # Check for diagonal mode in each sub-list
    if sub_lists[0] == sub_lists[1][::-1]:  # Check if the sub-list is a palindrome
        return True
    
    return False

In [246]:
count = 0
for i, row in dataset_enriched.iterrows():
    
    if has_diagonal_mode(row[13]):
        print(f"Diagonal pattern: {count} in this row {row[13]}, with id = {row[0]}")
        count += 1


Diagonal pattern: 0 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 39
Diagonal pattern: 1 in this row [2, 3, 3, 2, 3, 3, 2, 3, 3, 2], with id = 40
Diagonal pattern: 2 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 101
Diagonal pattern: 3 in this row [2, 2, 2, 2, 2, 2, 2, 2, 2, 2], with id = 159
Diagonal pattern: 4 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 388
Diagonal pattern: 5 in this row [3, 2, 2, 3, 3, 3, 3, 2, 2, 3], with id = 403
Diagonal pattern: 6 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 481
Diagonal pattern: 7 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 485
Diagonal pattern: 8 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 506
Diagonal pattern: 9 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 558
Diagonal pattern: 10 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 571


In [50]:
count = 0
for i, row in ind_col_dataset.iterrows():
    
    if has_diagonal_mode(row[10]):
        print(f"Diagonal pattern: {count} in this row {row[10]}, with id = {row[0]}")
        count += 1

        

Diagonal pattern: 0 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 18.0
Diagonal pattern: 1 in this row [2, 3, 3, 2, 3, 3, 2, 3, 3, 2], with id = 47.0
Diagonal pattern: 2 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 137.0
Diagonal pattern: 3 in this row [2, 2, 2, 2, 2, 2, 2, 2, 2, 2], with id = 169.0
Diagonal pattern: 4 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 31.0
Diagonal pattern: 5 in this row [3, 2, 2, 3, 3, 3, 3, 2, 2, 3], with id = 105.0
Diagonal pattern: 6 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 156.0
Diagonal pattern: 7 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 176.0
Diagonal pattern: 8 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 64.0
Diagonal pattern: 9 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 140.0
Diagonal pattern: 10 in this row [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], with id = 61.0


In [31]:
result1 = result[~result['id'].isin(to_be_eliminated)]